In [60]:
import pandas as pd
import seaborn

In [61]:
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable


In [62]:
# Data 로딩 ------------------------------------------------------------
df=seaborn.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [63]:
df.head(10)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False


In [64]:
# 성별, 등석, 나이에 따른 생존율 ----------------------------------------------------------------------------------
df['age'].mean(), df['age'].mode(), df['age'].value_counts()

(29.69911764705882,
 0    24.0
 Name: age, dtype: float64,
 24.00    30
 22.00    27
 18.00    26
 19.00    25
 28.00    25
          ..
 36.50     1
 55.50     1
 0.92      1
 23.50     1
 74.00     1
 Name: age, Length: 88, dtype: int64)

In [65]:
# 성별 생존율
sexGroup=df.groupby(by='sex')
sexGroup

In [66]:
# 성별 탑승객 수 체크
maleGroup=sexGroup.get_group('male')
femaleGroup=sexGroup.get_group('female')


len(maleGroup.index), len(femaleGroup.index)

(577, 314)

In [67]:
# 탑승객 생존자수 체크
sur=df.survived.sum()
sur

342

In [68]:
# 성별 탑승객 생존자 수 체크
mlGroup=maleGroup.groupby(by='alive')
flGroup=femaleGroup.groupby(by='alive')

ml=len(mlGroup.get_group('yes').index)
fl=len(flGroup.get_group('yes').index)

In [69]:
print('남녀 생존율')
print(f'남자 생존율은 {round((ml/sur)*100)}%',
      f'여자 생존율은 {round((fl/sur)*100)}%입니다.')

남녀 생존율
남자 생존율은 32% 여자 생존율은 68%입니다.


In [70]:
# 등석별 생존율
pclassGroup=df.groupby(by='pclass')
pclassGroup

In [71]:
# 등석별 탑승자 수 체크
oneGroup=pclassGroup.get_group(1)
twoGroup=pclassGroup.get_group(2)
threeGroup=pclassGroup.get_group(3)

len(oneGroup.index), len(twoGroup.index), len(threeGroup.index)

(216, 184, 491)

In [72]:
# 등석별 생존자 수 체크
olGroup=oneGroup.groupby(by='survived')
tlGroup=twoGroup.groupby(by='survived')
hlGroup=threeGroup.groupby(by='survived')

ol=len(olGroup.get_group(1).index)
tl=len(tlGroup.get_group(1).index)
hl=len(hlGroup.get_group(1).index)

ol, tl, hl

(136, 87, 119)

In [73]:
print('등석별 생존율')
print(f'1등석 생존율은 {round((ol/len(oneGroup.index))*100)}%',
      f'2등석 생존율은 {round((tl/len(twoGroup.index))*100)}%',
      f'3등석 생존율은 {round((hl/len(threeGroup.index))*100)}%입니다.', sep='\n')

등석별 생존율
1등석 생존율은 63%
2등석 생존율은 47%
3등석 생존율은 24%입니다.


In [74]:
# 나이별 생존율
# 결측치 처리
# 'age' 누락된 행 없애기
ageDF=df.dropna(how='any')

In [75]:
# 나이 구간 정하고 해당 구간 인원 수 리스트에 담기
ageGroup=ageDF.groupby(by='age')
age0_20=[]
age20_40=[]
age40_60=[]
age60_80=[]
for key, group in ageGroup:
    if key>=0.92 and key<20:
        age0_20.append(len(group['age'].index))
    elif key>=20 and key<40:
        age20_40.append(len(group['age'].index))
    elif key>=40 and key<60:
        age40_60.append(len(group['age'].index))
    else:
        age60_80.append(len(group['age'].index))
        
ageAll=sum(age0_20)+sum(age20_40)+sum(age40_60)+sum(age60_80) 
ageAll

182

In [76]:
# 나이별 생존자 수 체크
ageAlive=ageDF[ageDF['survived']==1]
ageAliveGroup=ageAlive.groupby(by='age')
ageAlive0_20=[]
ageAlive20_40=[]
ageAlive40_60=[]
ageAlive60_80=[]

for key, group in ageAliveGroup:
    if key>=0.92 and key<20:
        ageAlive0_20.append(len(group['age'].index))
    elif key>=20 and key<40:
        ageAlive20_40.append(len(group['age'].index))
    elif key>=40 and key<60:
        ageAlive40_60.append(len(group['age'].index))
    else:
        ageAlive60_80.append(len(group['age'].index))

In [77]:
print('나이별 생존율')
print(f'0세이상 20세미만 생존율은 {round((sum(ageAlive0_20)/sum(age0_20))*100)}%',
      f'20세이상 40세미만 생존율은 {round((sum(ageAlive20_40)/sum(age20_40))*100)}%',
      f'40세이상 60세미만 생존율은 {round((sum(ageAlive40_60)/sum(age40_60))*100)}%',
      f'60세이상 80세미만 생존율은 {round((sum(ageAlive60_80)/sum(age60_80))*100)}%입니다.', sep='\n')

나이별 생존율
0세이상 20세미만 생존율은 79%
20세이상 40세미만 생존율은 75%
40세이상 60세미만 생존율은 57%
60세이상 80세미만 생존율은 33%입니다.


# -----------------------------------------------------------